In [184]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import math

import pymorphy2
import re
from collections import defaultdict

from tqdm import tqdm, tqdm_notebook
from collections import namedtuple,defaultdict
import numpy as np

import sqlite3
import psycopg2


import time

#MS SQL 
import pyodbc

In [185]:
#MS SQL
server = 'NUTS' 
database = 'MK_news_2000'
username = '' 
password = '' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cur_sql = cnxn.cursor()

In [186]:
num = 509
cmd = "Select doc_text from Documents where id=?"
cur_sql.execute(cmd,num)
data = cur_sql.fetchone()[0]
data

'Все произошло 11 октября прошло года. Тогда расквартированный в Олонгапо 20-летний уроженец Массачусетса Джозеф Скотт Пембертон, являвшийся военнослужащим морской пехоты США, после службы решил зайти в местный бар. Там он познакомился с 26-летней Дженнифер, не зная, что девушка на самом деле мужчина-трансгендер, то есть олицетворяющий себя человеком другого пола. После этого молодые люди поехали в отель, сняли там номер и предались утехам. На суде морпех рассказал, что Дженнифер с отличием занималась с ним оральным сексом, а он провел у нее рукой между ног и тут обнаружил половой член. Американский военный, который даже в кошмарном сне не мог себе представить близость с другим мужчиной, настолько рассвирепел, что сначала придушил трансгендера, а затем завершил дело в туалете, утопив партнера в унитазе, передает Gaystarnews.com. 1 декабря 2015 года районный суд в Олонгапо Сити приговорил американца к 12 годам лишения свободы. Однако, адвокаты заявили, что он еще легко отделался. Ему гр

In [187]:
def f_sort(n, r):
    l = []
    for i in range(n):
        maxi = r.argmax()
        l.append(maxi)
        r[maxi] = 0
    return l

    
def parse_text(text):
    #print(type(text))
    words = (word for word in re.split('\W+', text) if (len(word) >0 )&( analyzer.normal_forms(word)[0].lower() not in stopWords ))
    lexems = (analyzer.normal_forms(word)[0] for word in words)
    
    return list(lexems)    
    
def s(l): # овзвр вес, но в случае просто суммируется
    weight = 0
    for word in l:
        weight+=mi_dict[new_word_dict[word]][0]
    return weight

def sent_weight(l,number_main_words):  #возвращает вес предложения
    weights=[]
    for word in l:
        weights.append(mi_dict[new_word_dict[word]][0])
    weights = np.array(weights)
    weight = np.sum(weights[:number_main_words])
    return weight

In [188]:
cmd = "select * from Doc_to_claster_probability where doc_id = ?"
cur_sql.execute(cmd,num)
distr=[]
for i in cur_sql.fetchone()[1:]:
    distr.append(i)
distr = np.array(distr)
claster = distr.argmax()





In [189]:
distr


array([  1.78176863e-08,   0.00000000e+00,   7.63817105e-03,
         1.06687917e-18,   4.73470268e-08,   1.12594666e-08,
         9.92361753e-01])

In [190]:
#MI_(num)
cmd = "select mi from MI_6"
cur_sql.execute(cmd)
MI = cur_sql.fetchall()

#словарь слово ключ : индекс слова, значение : вес
mi_dict={}
for i, mi in enumerate(MI):
    mi_dict[i] = mi

# переопределение словаря, теперь ключ: слово, значение :индекс
new_word_dict = {}
cmd = "select id, name from Words"
cur_sql.execute(cmd)
for i, word in tqdm(cur_sql.fetchall()):
    new_word_dict[word] = i 

100%|█████████████████████████████████| 38300/38300 [00:00<00:00, 77580.24it/s]


In [195]:
analyzer = pymorphy2.MorphAnalyzer() 
stopWords = stopwords.words('russian')
stopWords.extend(['что', 'это', 'так', 'вот','ещё', 'свой' , 'быть','й', 'как', 'в', '—', 'год','к', 'на','свой', 'который', 'е','кстати', 'также'])

sentences = sent_tokenize(data)
sentence_weight=[]
for i,sent in enumerate(sentences):
    parsed_text = parse_text(sent)
    sentence_weight.append(s(parsed_text))
    
sentence_weight = np.array(sentence_weight)
len(sentences)
x = f_sort(4,sentence_weight.copy())
x = sorted(x)
for i in x:
    print(i,sentences[i],len(word_tokenize(sentences[i])))


1 Тогда расквартированный в Олонгапо 20-летний уроженец Массачусетса Джозеф Скотт Пембертон, являвшийся военнослужащим морской пехоты США, после службы решил зайти в местный бар. 25
4 На суде морпех рассказал, что Дженнифер с отличием занималась с ним оральным сексом, а он провел у нее рукой между ног и тут обнаружил половой член. 29
5 Американский военный, который даже в кошмарном сне не мог себе представить близость с другим мужчиной, настолько рассвирепел, что сначала придушил трансгендера, а затем завершил дело в туалете, утопив партнера в унитазе, передает Gaystarnews.com. 40
9 Суд также обязал Пембертона выплатить семье жертвы $91,571 компенсации утраченного заработка, $3,291 расходов на погребение, $1,060 в качестве компенсации морального вреда и $636 в виде штрафных убытков. 34


In [183]:

        
N = 20 # кол-во слов для оценки  

sentences = sent_tokenize(data)
print("Number of sent = "+str(len(sentences)))
sentence_weight=[]
for n,sent in enumerate(sentences):
    parsed_text = parse_text(sent)
    if len(parsed_text) >N:      
        sentence_weight.append(sent_weight(parsed_text,N))
    else: 
        weight = 0
        for word in parsed_text:
            weight+=(mi_dict[new_word_dict[word]][0])
        sentence_weight.append(weight)
        
            
sentence_weight = np.array(sentence_weight)
x = f_sort(13,sentence_weight.copy())
x = sorted(x)
for i in x:
    print(i,sentences[i], len(word_tokenize(sentences[i])))        
        


Number of sent = 40
0 Как ранее писал «МК», не стоит ожидать от Трампа больше, чем он действительно способен сделать: так, например, если часть санкций в отношении России Трамп может отменить своим президентским указом, то для отмены другой части санкций требуется одобрение Конгресса — который вовсе не обязательно согласится. 53
1 Однако не все россияне, привыкшие к политической нестабильности последних двух лет, сразу могут сориентироваться: что принесет указ, принятый в заокеанских «верхах», простым жителям нашей страны? 33
2 О возможных ожиданиях рассказал в беседе с корреспондентом «МК» экономист Владислав Гинько: – Прежде всего нужно вспомнить слова Владимира Путина на одной из пресс-конференций — он сказал, что Россия готова отменить все контрсанкции при условии, что будут отменены именно санкции против нас. 48
4 Но напомним, что были приняты меры против ввоза некоторых продуктов питания из 28 стран Евросоюза, США, Канады, Австралии и Новой Зеландии — то есть вопрос еще и в том, 

In [116]:
a = sent_tokenize('С. С. С. Р. чемпиона')
a[0]

'С. С. С. Р. чемпиона'